<a href="https://colab.research.google.com/github/shu65/coffee-tuning/blob/main/coffee_tuning_blog%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install optuna gspread pandas

In [6]:
# Google スプレッドシートの承認

from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [29]:
# スプレッドシートからデータを取ってくる
import pandas as pd

ss_name = "shu65コーヒーデータ"
workbook = gc.open(ss_name)
worksheet = workbook.get_worksheet(1)
df = pd.DataFrame(worksheet.get_all_records())
df = df.set_index('淹れた日')
df

,豆,豆を買った店,豆を買った日,豆の量(g),ミルの時間 (sec),トータルの時間 (sec),蒸らし時間 (sec),スコア,コメント
淹れた日,,,,,,,,,
2020/12/29,マンデリン,店0,2020/12/28,9,5,240,30,5,マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象
2020/12/30,マンデリン,店0,2020/12/28,9,5,180,30,4,マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象


In [43]:

# 探索空間の定義

import optuna

search_space={
    "豆の量(g)": optuna.distributions.IntUniformDistribution(8, 12),
    "ミルの時間 (sec)": optuna.distributions.IntUniformDistribution(3, 15),
    "トータルの時間 (sec)": optuna.distributions.IntUniformDistribution(180, 300),
    "蒸らし時間 (sec)": optuna.distributions.IntUniformDistribution(20, 40),
    }
score_column = 'スコア'

In [44]:
# 現在までのデータをstudyに登録

import optuna

sampler = optuna.samplers.TPESampler(multivariate=True, seed=123)
study = optuna.create_study(direction='maximize', sampler=sampler)

for record_i, record in records.iterrows():
  print(record.to_dict())
  params = {}
  for key in search_space.keys():
    params[key] = record[key]
  trial = optuna.trial.create_trial(
    params=params,
    distributions=search_space,
    value=record[score_column])
  study.add_trial(trial)

/usr/local/lib/python3.6/dist-packages/optuna/samplers/_tpe/sampler.py:191: ExperimentalWarning:

``multivariate`` option is an experimental feature. The interface can change in the future.

[I 2020-12-29 07:06:43,450] A new study created in memory with name: no-name-5fe4bdce-09ef-4507-9189-aa3cb82eb317


{'豆': 'マンデリン', '豆を買った店': '店0', '豆を買った日': '2020/12/28', '豆の量(g)': 9, 'ミルの時間 (sec)': 5, 'トータルの時間 (sec)': 180, '蒸らし時間 (sec)': 30, 'スコア': 4, 'コメント': 'マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象'}
{'豆': 'マンデリン', '豆を買った店': '店0', '豆を買った日': '2020/12/28', '豆の量(g)': 9, 'ミルの時間 (sec)': 5, 'トータルの時間 (sec)': 240, '蒸らし時間 (sec)': 30, 'スコア': 5, 'コメント': 'マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象'}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: ExperimentalWarning:

create_trial is experimental (supported from v2.0.0). The interface can change in the future.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: ExperimentalWarning:

add_trial is experimental (supported from v2.0.0). The interface can change in the future.



In [45]:
# 次のパラメータの出力
trial = study._ask()

new_params = {}
for key, space in search_space.items():
  new_params[key] = trial._suggest(key, space)

for key in  ["豆の量(g)", "ミルの時間 (sec)",  "トータルの時間 (sec)",  "蒸らし時間 (sec)"]:
  print(key, new_params[key])

豆の量(g) 10
ミルの時間 (sec) 15
トータルの時間 (sec) 278
蒸らし時間 (sec) 26
